In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mnist_example import MNISTModel, MNISTDataModule

datamodule = MNISTDataModule(batch_size=10)
datamodule.setup()

len(datamodule.test_dataloader())

10

In [3]:
from energizer.data.datamodule import ActiveDataModule

dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

dm.is_on_pool = False
print(len(dm.test_dataloader()))
dm.is_on_pool = True
print(len(dm.test_dataloader()))

10
10


In [4]:
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )
# dm.label(list(range(100)))
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )

---

In [5]:
from energizer.learners.base import Learner
from energizer.trainer import Trainer
import torch
from torch.utils.data import DataLoader
from energizer.learners.base import Deterministic
from energizer.learners.acquisition_functions import entropy
from pytorch_lightning import LightningModule, Trainer as pl_Trainer

In [6]:
class ActiveLearner(Deterministic):

    def pool_step(self, batch, *args, **kwargs) -> torch.Tensor:
        x, _ = batch
        logits = self(x)
        scores = entropy(logits)
        return scores

In [7]:
model = MNISTModel()
active_learner = ActiveLearner(model)

In [8]:
pl_trainer = pl_Trainer(max_epochs=1)
pl_trainer.fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:589: UserWarning: You have overridden `on_before_batch_transfer` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.
  f"You have overridden `{hook_name}` in `LightningModule` but have passed in a"
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:589: UserWarning: Yo

Epoch 0: 100%|██████████| 20/20 [00:00<00:00, 63.70it/s, loss=2.33, v_num=329]


In [9]:
pl_trainer.test(active_learner, datamodule)

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 230.57it/s]


[{}]

In [10]:
trainer = Trainer(query_size=2, max_epochs=1, max_labelling_iters=2)
trainer.active_fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2880, 2.2842, 2.2846, 2.2863, 2.2904, 2.2851, 2.2881,
        2.2840, 2.2930, 2.2916])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2930, 2.2916])
top_indices: tensor([8, 9]) 

current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2930, 2.2916, 2.2906, 2.2885, 2.2944, 2.2849, 2.2875, 2.2950, 2.2904,
        2.2890, 2.2841, 2.2902])
all_indices: tensor([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
top_scores: tensor([2.2950, 2.2944])
top_indices: tensor([15, 12]) 

current batch_size: 10
current_indices: tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
size: tensor(30)
all_scores: tensor([2.2950, 2.2944, 2.2894, 2.2831, 2.2916, 2.2906, 2.2934, 2.2829, 2.2874,
        2.2923, 2.2918, 2.2928])
all_indices: tensor([15, 12, 20, 21, 22, 

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


current batch_size: 10
current_indices: tensor([70, 71, 72, 73, 74, 75, 76, 77, 78, 79])
size: tensor(80)
all_scores: tensor([2.2926, 2.2923, 2.2794, 2.2906, 2.2858, 2.2862, 2.2882, 2.2893, 2.2887,
        2.2907, 2.2899, 2.2862])
all_indices: tensor([47, 23, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79])
top_scores: tensor([2.2926, 2.2923])
top_indices: tensor([47, 23]) 

current batch_size: 10
current_indices: tensor([80, 81, 82, 83, 84, 85, 86, 87, 88, 89])
size: tensor(90)
all_scores: tensor([2.2926, 2.2923, 2.2856, 2.2857, 2.2845, 2.2861, 2.2896, 2.2800, 2.2876,
        2.2862, 2.2893, 2.2907])
all_indices: tensor([47, 23, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89])
top_scores: tensor([2.2926, 2.2923])
top_indices: tensor([47, 23]) 

compute indices: tensor([47, 23])
Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 190.57it/s]
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2880, 2.2842, 2.2846,

...

Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 171.87it/s]
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2842, 2.2846, 2.2863, 2.2851, 2.2840, 2.2849, 2.2875,
        2.2841, 2.2831, 2.2829])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2875, 2.2863])
top_indices: tensor([6, 2]) 

current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2875, 2.2863, 2.2874, 2.2793, 2.2848, 2.2872, 2.2872, 2.2854, 2.2809,
        2.2864, 2.2831, 2.2870])
all_indices: tensor([ 6,  2, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
top_scores: tensor([2.2875, 2.2874])
top_indices: tensor([ 6, 10]) 

current batch_size: 10
current_indices: tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
size: tensor(30)
all_scores: tensor([2.2875, 2.2874, 2.2876, 2.2826, 2.2852, 2.2859, 2.2810, 2.2863, 2.2870,
   

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


tensor([20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
size: tensor(30)
all_scores: tensor([2.2863, 2.2863, 2.2794, 2.2858, 2.2862, 2.2862, 2.2856, 2.2857, 2.2845,
        2.2861, 2.2800, 2.2862])
all_indices: tensor([18,  2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])
top_scores: tensor([2.2863, 2.2863])
top_indices: tensor([18,  2]) 

compute indices: tensor([18,  2])
Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 196.66it/s]
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2842, 2.2846, 2.2851, 2.2840, 2.2849, 2.2841, 2.2831,
        2.2829, 2.2793, 2.2848])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2851, 2.2849])
top_indices: tensor([2, 4]) 

current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2851, 2.2849, 2.2854, 2.2809, 2.2831, 2.2826, 2.2852, 2.2859, 2.2810,
        2

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1931: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


current batch_size: 10
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
size: tensor(20)
all_scores: tensor([2.2851, 2.2849, 2.2854, 2.2809, 2.2831, 2.2826, 2.2852, 2.2810, 2.2794,
        2.2856, 2.2845, 2.2800])
all_indices: tensor([ 2,  4, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
top_scores: tensor([2.2856, 2.2854])
top_indices: tensor([17, 10]) 

compute indices: tensor([17, 10])
Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 200.13it/s]
current batch_size: 10
current_indices: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
size: tensor(10)
all_scores: tensor([  -inf,   -inf, 2.2842, 2.2846, 2.2851, 2.2840, 2.2849, 2.2841, 2.2831,
        2.2829, 2.2793, 2.2848])
all_indices: tensor([-1, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9])
top_scores: tensor([2.2851, 2.2849])
top_indices: tensor([2, 4]) 

current batch_size: 8
current_indices: tensor([10, 11, 12, 13, 14, 15, 16, 17])
size: tensor(18)
all_scores: tensor([2.2851, 2.2849, 2.2809, 2.2831, 2.2826, 2.2852, 2.

In [ ]:
test_dl = DataLoader(
    [datamodule.mnist_test[i] for i in range(10)], 
    shuffle=False, 
    batch_size=2,
)
x, y = next(iter(test_dl))

In [ ]:
active_learner(x).argmax(-1)

In [ ]:
y